# State Farm Distracted Driver Detection

In [19]:
# %pip install ultralytics
# %pip install split-folders
# %pip install -U ipywidgets

In [20]:
import csv
import os
from glob import glob
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from ultralytics import YOLO
import splitfolders
import pandas as pd
import numpy as np
from IPython.display import display, Image
from sklearn.metrics import accuracy_score

## Loading the data

In [21]:
data = {}

with open('../datasets/state-farm-distracted-driver-detection/driver_imgs_list.csv') as file:
    read_file = csv.reader(file)
    read_file = list(read_file)
    
    for row in read_file[1:]:
        key = row[1]
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[2]]

In [22]:
data['c0'][:5]

['img_44733.jpg',
 'img_72999.jpg',
 'img_25094.jpg',
 'img_69092.jpg',
 'img_92629.jpg']

In [23]:
classes_list = list(data.keys())
classes_list

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [24]:
dataset_folder = '../datasets/state-farm-distracted-driver-detection/imgs/'

train_dir = os.path.join(dataset_folder, 'train/')
test_dir = os.path.join(dataset_folder, 'test/')

In [25]:
print('Number of images in the training dataset : ', str(len(glob(train_dir+'*/*'))))
print('Number of images in the testing dataset : ', str(len(glob(test_dir+'*'))))

Number of images in the training dataset :  22424
Number of images in the testing dataset :  79726


In [ ]:
# Need to separate the training set and the validation set.

### Writing helper function for creating directories for training set, validation set and test set

In [26]:
def remove_directory(path):
    for root, dirs, files in os.walk(path, topdown = False):
        for name in files:
            file_path = os.path.join(root, name)
            os.remove(file_path)
        for name in dirs:
            dir_path = os.path.join(root, name)
            os.rmdir(dir_path)
    os.rmdir(path)

In [27]:
def create_directories(paths, subfolders):
    for path in paths:
        if os.path.exists(path):
            remove_directory(path)
        
        for folder in subfolders:
            subfolder_path = os.path.join(path, folder)
            os.makedirs(subfolder_path)

In [28]:
paths = ['../datasets/state-farm-distracted-driver-detection/cleaned_dataset/train',
         '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/val',
        '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/test']

In [29]:
subfolders = classes_list

### Creating Train, Val, Test folders along with sub-directories (all Classes)

In [31]:
create_directories(paths, subfolders)

### Creating the cleaned dataset using the above helper functions we have created

In [32]:
split_size = [0.6, 0.2]


for clas, images in data.items():
    # print(len(images))
    train_size = int(split_size[0]*len(images))
    # print("Train size: ", train_size)
    
    test_size = int(split_size[1]*len(images))
    #print("Test size: ", test_size)
    
    train_images = images[:train_size]
    # print("Train Images Length", len(train_images))
    
    val_images = images[train_size: train_size + test_size]
    # print("Val Images Length", len(val_images))
    
    test_images = images[train_size + test_size:]
    # print("Test Images Length", len(test_images))
    
    
    
    for image in train_images:
        source = os.path.join(train_dir, clas, image)
        # print(os.path.exists(source))
        dest = os.path.join(paths[0], clas, image)
        copyfile(source, dest)
    
    for image in val_images:
        source = os.path.join(train_dir, clas, image)
        dest = os.path.join(paths[1], clas, image)
        copyfile(source, dest)
    
    for image in test_images:
        source = os.path.join(train_dir, clas, image)
        dest = os.path.join(paths[2], clas, image)
        copyfile(source, dest)
    
    


### Using a better approach for creating the cleaned dataset using `splitfolders` module

### First deleting the cleaned dataset created usinfg the above method

In [33]:
remove_directory('../datasets/state-farm-distracted-driver-detection/cleaned_dataset')

### Creating the cleaned dataset now using splitfolder module

In [34]:
images_dir = '../datasets/state-farm-distracted-driver-detection/imgs/train'
output_folder = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset' # Note: the function will create val, train, test sub directories by itself
split_ratio = (0.6, 0.2, 0.2)



splitfolders.ratio(images_dir, output= output_folder, seed = 10, ratio= split_ratio)

Copying files: 22424 files [00:03, 5949.75 files/s]


Done ! Just needed one line of code.

### From now, we will be using these Directory paths for our training, validation and testing purpose

In [35]:
train_dir = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/train'
val_dir = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/val'
test_dir = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/test'

## Creating Image data generator Function with Data Augmentation

In [46]:
def imagedatageneration(train_dir, val_dir, test_dir, target_size = (256, 256), batch_size = 32):
    train_datagen = ImageDataGenerator(rescale = 1.0 / 255,
                                       rotation_range = 30,
                                       width_shift_range = 0.1,
                                       height_shift_range = 0.1,
                                       zoom_range = 0.1,
                                       shear_range = 0.1,
                                       fill_mode = "nearest"
                                      )
    train_generator = train_datagen.flow_from_directory(
                                                            train_dir,
                                                            target_size = target_size,
                                                            class_mode = 'categorical',
                                                            shuffle = True,
                                                            batch_size = batch_size
                                                        )
    
    
    val_datagen = ImageDataGenerator(rescale = 1.0 / 255)
    val_generator = val_datagen.flow_from_directory(
                                                        val_dir,
                                                        target_size = target_size,
                                                        class_mode = 'categorical',
                                                        shuffle = True,
                                                        batch_size = batch_size
                                                    )
    
    test_datagen = ImageDataGenerator(rescale = 1.0/255)
    
    test_generator = test_datagen.flow_from_directory(
                                                        test_dir,
                                                        target_size = target_size,
                                                        class_mode = 'categorical',
                                                        shuffle = False,
                                                        batch_size = 1
                                                      )
    
    return train_generator, val_generator, test_generator
    

In [47]:
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01)

In [ ]:
# train_dir = '../datasets/state-farm-distracted-driver-detection/train'
# val_dir = '../datasets/state-farm-distracted-driver-detection/val'

train_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
                                                        train_dir,
                                                        target_size = (256, 256),
                                                        class_mode = 'categorical',
                                                        shuffle = True,
                                                        batch_size = 32
                                                    )

val_datagen = ImageDataGenerator(rescale = 1.0/255)

val_generator = val_datagen.flow_from_directory(
                                                        val_dir,
                                                        target_size = (256, 256),
                                                        class_mode = 'categorical',
                                                        shuffle = True,
                                                        batch_size = 32
                                                )



## First Model -> Dense Model

In [48]:
train_generator, val_generator, test_generator = imagedatageneration(train_dir, val_dir, test_dir)

Found 13451 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.
Found 4492 images belonging to 10 classes.


In [49]:
model1 = tf.keras.models.Sequential([
    Flatten(input_shape = (256, 256, 3)),
    Dense(1024, activation = 'relu'),
    BatchNormalization(),
    Dense(512, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
])

In [50]:
model1.compile(optimizer = Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['acc'])

In [51]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 196608)            0         
                                                                 
 dense (Dense)               (None, 1024)              201327616 
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 256)               1

In [ ]:
model1.fit(train_generator,
                   epochs = 10,
                   verbose = 1,
                   validation_data = val_generator,
                   callbacks = [es])

Epoch 1/10
315/421 [=====================>........] - ETA: 25s - loss: 2.3069 - acc: 0.1891

## Second Model -> CNN Model

In [ ]:
model2 = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 3)),
    Conv2D(32, (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation = 'relu'),
    Conv2D(64, (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation = 'relu'),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
])

In [ ]:
model2.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['acc'])
model2.summary()

In [ ]:
model2.fit(train_generator,
          epochs = 20,
          verbose = 1,
          validation_data = val_generator,
          callbacks = [es])

## Third Model -> VGG16

In [ ]:
pretrained_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))
pretrained_model.summary()

In [ ]:
# for layer in pretrained_model.layers[:-5]:
#     layer.trainable = False
    

In [ ]:
last_layer = pretrained_model.get_layer('block5_pool')
last_output = last_layer.output

model3 = tf.keras.models.Sequential([ 
    pretrained_model,
    Flatten(),
    Dense(2048, activation = 'relu'),
    Dense(1024, activation = 'relu'),
    Dropout(0.2),
    Dense(256, activation = 'relu'),
    Dropout(0.2),
    Dense(10, activation = 'softmax')
    
])

In [ ]:
model3.compile(optimizer = Adam(learning_rate = 0.005), loss = 'categorical_crossentropy', metrics = ['acc'])
model3.summary()

In [ ]:
model3.fit(train_generator,
          steps_per_epoch = 250,
          epochs = 20,
          verbose = 1,
          validation_steps = 50,
          validation_data = val_generator,
          callbacks = [es])

## Fourth Model -> ResNet50

In [ ]:
pretrained_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))

In [ ]:
for layer in pretrained_model.layers[:-3]:
    layer.trainable = False

In [ ]:
model4 = tf.keras.models.Sequential([ 
    pretrained_model,
    Flatten(),
    Dense(512, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
])

In [ ]:
model4.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'categorical_crossentropy', metrics = ['acc'])
model4.summary()

In [ ]:
model4.fit(train_generator,
          steps_per_epoch = 250,
          epochs = 20,
          verbose = 1,
          validation_steps = 50,
          validation_data = val_generator,
          callbacks = [es])

## Fifth Model -> Yolo v9

In [ ]:
model5 = YOLO('yolov8n-cls.pt')

In [ ]:
splitfolders.ratio("../datasets/state-farm-distracted-driver-detection/imgs/train", output="../datasets/state-farm-distracted-driver-detection/output", seed = 1337, ratio=(0.7, 0.15, 0.15))

In [ ]:
results = model5.train(data = "../datasets/state-farm-distracted-driver-detection/output", epochs = 10)

In [ ]:
model5.val()

In [ ]:
df= pd.read_csv('./runs/classify/train2/results.csv')
df.head()

In [ ]:
Image("./runs/classify/train2/results.png")

In [ ]:
Image("./runs/classify/train2/confusion_matrix_normalized.png")


In [ ]:
# classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
# path = "../datasets/state-farm-distracted-driver-detection/output/test/c1/"
# actual_class = 1
# model_weights = "./runs/classify/train2/weights/best.pt"
# pred = [(path+i,model5.predict(path+i, model = model_weights)[0].probs.top1, actual_class) for i in os.listdir(path)[:45]]

In [ ]:
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
test_images_path = "../datasets/state-farm-distracted-driver-detection/output/test/"
model_weights = "./runs/classify/train2/weights/best.pt"

predicted_list = []

for clas in classes:
    image_dir = os.path.join(test_images_path, clas)
    # print(image_dir)
    images_list = os.listdir(image_dir)
    # print(images_list)
    # Class label in the form of 0 to 9
    class_label = int(clas[-1])
    # print(class_label)
    for image in images_list:
        path = os.path.join(image_dir, image)
        # print(path)
        y_actual = class_label
        y_predicted = model5.predict(path, model = model_weights)[0].probs.top1
        predicted_list.append([path, y_actual, y_predicted])

In [ ]:
print("length of the Predicted List : ", len(predicted_list))

In [ ]:
df = pd.DataFrame(predicted_list, columns = ['Image_path', 'Y_actual', 'Y_predicted'])

In [ ]:
accuracy_score(df['Y_actual'], df['Y_predicted'])